# Processamento do Arquivo de Vacinação - Recente

**Processamento dos arquivos a partir de 04/05**

*A partir de 24/05, a secretaria incluiu novos grupos na vacinação. Por conta disso, foi criado um novo notebook para processamento. Verifique no README o documento correto para o processamento dos arquivos posterior a 24/05/21.*

Este documento é utilizado para processar o arquivo **XLSX** disponibilizado pela **Secretaria de Estado da Saúde do estado de Sergipe** com os quantitativos da vacinação do estado. Foi utilizada a biblioteca **[pandas](https://pandas.pydata.org/)** para carregar e processar o arquivo com a finalidade de gerar um arquivo .csv mais adequado para o processo de visualização de dados. Um exemplo de visualização destes dados está no Painel COVID-19 disponível [neste link](https://covid19.data2learning.com).

Diferentes tipos de arquivos já foram compartilhados pela secretaria. Esse processamento se refere ao arquivo disponibilizado a partir de **04/05/21**. No dia 04/05 teve uma nova mudança no arquivo do Excel com a inclusão de novas colunas referente a novos grupos de vacionados: puerperas com comobidade, gestante com comorbidade, deficiente permanente, comorbidades.

Estrutura do arquivo disponibilizado a partir de 04/05:

* Municípios
* ESTIMATIVA POPULACIONAL
    * População Total
    * 60 a 69 anos
    * 70 a 79 anos
    * 80 anos e mais
* DOSES ENVIADAS
    * Total da 1ª dose enviada
    * Total da 2ª dose enviada
* DOSES APLICADAS
    * Total da 1ª dose aplicada
    * Total da 2ª dose aplicada
* DOSES APLICADAS POR GRUPO
    * Trabalhadores de Saúde
        * 1ª dose
        * 2ª dose
    * Pessoas com 60 anos ou mais e deficientes institucionalizados
        * 1ª dose
        * 2ª dose
    * Indígenas aldeados
        * 1ª dose
        * 2ª dose
    * Quilombolas
        * 1ª dose
        * 2ª dose
    * Trabalhadores das forças de segurança e Forças Armadas
        * 1ª dose
        * 2ª dose
    * Pessoas com Síndrome de Dowm e Autismo
        * 1ª dose
        * 2ª dose
    * Outros
        * 1ª dose
        * 2ª dose
    * Situação de Rua
        * 1ª dose
        * 2ª dose
    * 60 a 69 anos
        * 1ª dose
        * 2ª dose
    * 70 a 79 anos
        * 1ª dose
        * 2ª dose
    * 80 anos e mais 
        * 1ª dose
        * 2ª dose
    * Puerperas com comorbidade
        * 1ª dose
    * Gestante com comorbidade
        * 1ª dose
    * Deficiente Permanente
        * 1ª dose
    * Comorbidades
        * 1ª dose
* COBERTURA VACINAL
    * PRIMEIRA DOSE
        * População Geral
        * 60 a 69 anos
        * 70 a 79 anos
        * 80 anos e mais
        * % 1ª dose aplicada
    * SEGUNDA DOSE
        * População Geral
        * 60 a 69 anos
        * 70 a 79 anos
        * 80 anos e mais
        * % 2ª dose aplicada

Totalizando 45 colunas com informações de população, doses enviadas, doses aplicadas e cobertura. 




## Etapas do Processamento - EXCEL

Para trabalhar melhor com as informações, foi preciso fazer um organização das informações disponbilizadas. Apesar do pandas lê facilmente arquivos do Excel, a estrutura em níveis das colunas no documento dificulta um pouco o trabalho no DataFrame gerado. Além disso, o documento possui algumas linhas com informações de título do documento como descrição do Governo, secretaria e coordenação responsável pelo documento. Como a idéia é gerar um arquivo .CSV que seja de fácil leitura, essas informações precisam ser removidas do documento.

As etapas do processamento foram: 

1. Definir uma nomeclatura para cada coluna para excluir a hierarquia criada no arquivo excel. 
2. Definir as colunas de interesse para geração do CSV. 
3. Carregar o XLSX e gerar o arquivo final.

Segue a execução das etapas. 

In [36]:
# Import da biblioteca pandas

import pandas as pd
import numpy as np

### Nomeclatura das colunas

Algumas colunas forma renomeadas de acordo com os grupos de vacinação aptos para vacinação, conforme o arquivo do excel disponibilizado. São eles:

* Grupo 1: Trabalhadores da Saúde
* Grupo 2: Pessoas com 60 anos ou mais e deficientes Institucionalizados
* Grupo 3: Índio Aldeado
* Grupo 4: Pessoas com 80 anos e mais
* Grupo 5: Pessoas de 70 a 79 anos 
* Grupo 6: Pessoas de 60 a 69 anos
* Grupo 7: Situação de Rua
* Grupo 8: Quilombolas
* Grupo 9: Trabalhadores das forças de segurança
* Grupo 10: Pessoas com Síndrome de Dowm e Autismo
* Grupo 11: Outros (População vacinada com resíduos de vacinas ao final do dia) 
* Grupo 12: Puerperas com comorbidade
* Grupo 13: Gestante com comorbidade
* Grupo 14: Deficiente Permanente
* Grupo 15: Comorbidades

Com isso foram definidos  os nomes de cada coluna de acordo com o que estava disponível no arquivo original para cada grupo. Além disso, o arquivo contém informações  sobre a estimativa populacional de algunas grupos, os totais de doses aplicadas e a cobertura vacinal. 

In [37]:
# Lista com os nomes das colunas do documento excel.

list_columns = [
    'municipio',
    'populacao_total',
    'populacao_grupo6',
    'populacao_grupo5',
    'populacao_grupo4',
    'enviadas_dose1',
    'enviadas_dose2',
    'aplicadas_dose1',
    'aplicadas_dose2',
    'aplicadas_dose1_grupo1',
    'aplicadas_dose2_grupo1',
    'aplicadas_dose1_grupo2',
    'aplicadas_dose2_grupo2',
    'aplicadas_dose1_grupo3',
    'aplicadas_dose2_grupo3',
    'aplicadas_dose1_grupo8',
    'aplicadas_dose2_grupo8',
    'aplicadas_dose1_grupo9',
    'aplicadas_dose2_grupo9',
    'aplicadas_dose1_grupo10',
    'aplicadas_dose2_grupo10',
    'aplicadas_dose1_grupo11',
    'aplicadas_dose2_grupo11',
    'aplicadas_dose1_grupo7',
    'aplicadas_dose2_grupo7',
    'aplicadas_dose1_grupo6',
    'aplicadas_dose2_grupo6',
    'aplicadas_dose1_grupo5',
    'aplicadas_dose2_grupo5',
    'aplicadas_dose1_grupo4',
    'aplicadas_dose2_grupo4',
    'aplicadas_dose1_grupo12',
    'aplicadas_dose1_grupo13',
    'aplicadas_dose1_grupo14',
    'aplicadas_dose1_grupo15',
    'cobertura_dose1_populacao_geral',
    'cobertura_dose1_grupo6',
    'cobertura_dose1_grupo5',
    'cobertura_dose1_grupo4',
    'cobertura_dose1',
    'cobertura_dose2_populacao_geral',
    'cobertura_dose2_grupo6',
    'cobertura_dose2_grupo5',
    'cobertura_dose2_grupo4',
    'cobertura_dose2'
]

### Colunas de interesse 

Nem todas as colunas foram incluídas no CSV final. As colunas referente a cobertura possui informação em %. Em alguns testes que foram feitos, o pandas não gerava essa coluna de forma adequada quando era carregado. Como essa é uma informação fácil de calcular pela relação entre as doses envidas e aplicadas, foi preferido retira-la do documento final. No processo de visualização essa informação pode ser facilmente calculada e exibida. Isso é feito no Painel linkado no início do documento. A linha com o total do estado de Sergipe também foi retirada já que isso pode ser calculado a partir dos dados das colunas anteriores.

In [38]:
# Colunas definidas para o documento final

final_columns = [
    'codigo_municipio',
    'municipio',
    'populacao_total',
    'populacao_grupo4',
    'populacao_grupo5',
    'populacao_grupo6',
    'enviadas_dose1',
    'enviadas_dose2',
    'aplicadas_dose1',
    'aplicadas_dose2',
    'aplicadas_dose1_grupo1',
    'aplicadas_dose1_grupo2',
    'aplicadas_dose1_grupo3',
    'aplicadas_dose1_grupo4',
    'aplicadas_dose1_grupo5',
    'aplicadas_dose1_grupo6',
    'aplicadas_dose1_grupo7',
    'aplicadas_dose1_grupo8',
    'aplicadas_dose1_grupo9',
    'aplicadas_dose1_grupo10',
    'aplicadas_dose1_grupo11',
    'aplicadas_dose1_grupo12',
    'aplicadas_dose1_grupo13',
    'aplicadas_dose1_grupo14',
    'aplicadas_dose1_grupo15',
    'aplicadas_dose2_grupo1',
    'aplicadas_dose2_grupo2',
    'aplicadas_dose2_grupo3',
    'aplicadas_dose2_grupo4',
    'aplicadas_dose2_grupo5',
    'aplicadas_dose2_grupo6',
    'aplicadas_dose2_grupo7',
    'aplicadas_dose2_grupo8',
    'aplicadas_dose2_grupo9',
    'aplicadas_dose2_grupo10',
    'aplicadas_dose2_grupo11',
]

### Carregar o XLSX e gerar o arquivo final

Os trechos de códigos a seguir servem para carregar o arquivo .xslx e gerar o arquivo CSV final. Como já definimos os nomes das colunas manualmente, podemos descartar as 3 primeiras linhas do documento que possui informações de título e nome das colunas de forma hierárquica. A explicação de cada etapa está nos comentários. 

Foi necessário aplicar a separação das informações na coluna Município, já que é passado código e nome do município em uma só colunda. No documento final serão geradas duas colunas distintas para essa informação: codigo_municipio e municipio.

In [69]:
data_ = "2021-05-22"
data_temp = data_.replace("-","")

'''
Carrega o arquivo .xlsx descartando as 3 primeiras linhas (skiprows=8) que correspondem
as linhas de informações títulos das colunas. O parâmetro names força que 
o dataframe gerado tenha os nomes das colunas definidas manualmente. 
''' 
data_final = pd.read_excel("data/" + data_ + ".xlsx", skiprows=2,names=list_columns)

'''
.head(-1) exclui a última linha do documento que possui o total de Sergipe. Essa informação
foi retirada porque é facilmente calcula a partir da soma de cada coluna. Isso pode ser feito
no momento de gerar as visualizações

Use -10 de 11/03 até 23/03
Use -9 a partir de 23/03
Use -11 a partir de 29/03
Use -12 em 16/04
Use -11 a partir em 17/04
USE -10 a partir de 20/04
USE -11 em 24/04
USE -10 a partir de 25/04

''' 
data_final = data_final.head(-10)

'''
Separando a coluna município em duas novas colunas: codigo_municipio e municipio
'''
data_final['codigo_municipio'] = data_final['municipio'].str[0:6]
data_final['municipio'] = data_final['municipio'].str[7:]


'''
Gera o datframe com as colunas selecinadas para o documento final 
'''
data_final = data_final[final_columns]

'''
Alguns municípios possui valores em branco para algumas colunas. Quando o dataframe é carregado,
esses valores são reconhecidos como NULOS. O código a seguir substitui por 0.
'''
data_final = data_final.fillna(0)

'''
Força que as colunas que possui informações da vacinação seja do tipo inteiro.
final_columns[1:] pega da segunda coluna até a última, que são as colunas do tipo inteiro. 
'''
data_final[final_columns[2:]] = data_final[final_columns[2:]].round().astype(int)

'''
Visualiza o dataframe final 
''' 
data_final

,codigo_municipio,municipio,populacao_total,populacao_grupo4,populacao_grupo5,populacao_grupo6,enviadas_dose1,enviadas_dose2,aplicadas_dose1,aplicadas_dose2,...,aplicadas_dose2_grupo2,aplicadas_dose2_grupo3,aplicadas_dose2_grupo4,aplicadas_dose2_grupo5,aplicadas_dose2_grupo6,aplicadas_dose2_grupo7,aplicadas_dose2_grupo8,aplicadas_dose2_grupo9,aplicadas_dose2_grupo10,aplicadas_dose2_grupo11
0,280010,Amparo de São Francisco,2380,51,89,140,642,232,539,213,...,0,0,26,84,37,0,0,9,0,0
1,280020,Aquidabã,21681,481,932,1543,4544,2314,3527,2047,...,17,0,163,769,703,0,0,15,0,0
2,280030,Aracaju,664908,10149,24138,47854,149225,66350,141802,48593,...,339,0,2493,14557,9260,43,50,590,211,0
3,280040,Arauá,9947,149,348,644,1812,942,1760,966,...,0,0,111,465,220,0,0,0,8,0
4,280050,Areia Branca,18686,281,548,1047,3071,1521,2584,1361,...,0,0,147,555,393,0,0,10,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,280720,Siriri,8970,154,316,533,1758,973,1527,905,...,0,0,157,297,306,0,0,9,1,0
71,280730,Telha,3249,49,100,178,589,339,600,334,...,0,0,27,101,114,0,0,6,4,0
72,280740,Tobias Barreto,52530,1046,2188,3513,10363,5838,9562,5248,...,33,0,457,2023,1658,0,0,88,33,0
73,280750,Tomar do Geru,13535,208,446,760,2206,1136,1659,1001,...,0,0,131,376,350,0,0,9,0,0


O código a seguir gera o arquivo final no formato CSV. O atributo `index=False` força que o documento não tenha um índice por linha. Essa informação, neste caso, não é necessária para os processos de visualização.

In [70]:
#data_final.to_csv("output/relatorio_vacina_sergipe.csv", index=False)
data_final.to_csv("output/historico/" + data_temp + ".csv", index=False)

Desenvolvido por **@adolfoguimaraes**.

Qualquer dúvida sobre o código, só entrar em contato: 
* Twitter: https://twitter.com/adolfoguimaraes
* Instagram: https://instagram.com/profadolfoguimaraes
* GitHub: https://github.com/adolfoguimaraes

*Atenção: Esse trabalho é feito de forma totalmente independente e não tem nenhuma ligação com a secretaria que disponibiliza os dados.*